In [1]:
import pandas as pd

c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
import os

In [3]:
import numpy as np

In [4]:
import matplotlib.pyplot as plt

In [5]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, confusion_matrix

In [6]:
gt_map = dict()

In [7]:
for subdir, dirs, files in os.walk("./SCORE_SKAB/GT"):
    for file in files:
        if "npy" not in file:
            fpath = os.path.join(subdir, file)
            
            df = pd.read_csv(fpath, header = None)
            values = [x[0] for x in df.values]
            
            fname = file.split(".csv")[0]
            
            gt_map[fname] = values.copy()

In [8]:
scores_map = dict()
rocauc_map = dict()

In [9]:
def compute_threshold(in_data, th_type, th_factor):
    if th_type == "MV":
        return np.max(in_data)
    elif th_type == "MAD":
        return np.median(in_data) + 1.4826*th_factor*np.median(np.abs(in_data - np.median(in_data)))
    elif th_type == "SD":
        return np.mean(in_data) + th_factor*np.std(in_data)
    elif th_type == "IQR":
        return np.percentile(in_data, 75) + th_factor*(np.percentile(in_data, 75) - np.percentile(in_data, 25))

In [10]:
def compute_rocauc(pred, gt):
    if len(pred) > len(gt):
        pred = pred[-len(gt):]
    elif len(pred) < len(gt):
        gt = gt[-len(pred):]
        
    return roc_auc_score(gt, pred)

In [11]:
val_scores = dict()

In [12]:
anomaly_scores_by_method = dict()

In [13]:

for subdir, dirs, files in os.walk("VAL_SKAB"):

    for file in files:
        
        arch = subdir.split("\\")[1]
        
        if arch not in val_scores:
            val_scores[arch] = dict()
            
        if "npy" in file:
            fpath = os.path.join(subdir, file)
            
            values = [x for x in np.load(fpath)]
            if(arch=='USAD'):
                values=values[300:350]
            fname = file.split(".npy")[0]
            
            val_scores[arch][fname] = values

        if "csv" in file:
            fpath = os.path.join(subdir, file)
           
            df = pd.read_csv(fpath, header = None)
            values = [x[0] for x in df.values]

            fname = file.split(".csv")[0]
            val_scores[arch][fname] = values



In [14]:

for subdir, dirs, files in os.walk("SCORE_SKAB"):

    for file in files:
        
        arch = subdir.split("\\")[1]
        
        if arch not in val_scores:
            val_scores[arch] = dict()
            
        if "npy" in file:
            fpath = os.path.join(subdir, file)
            
            values = [x for x in np.load(fpath)]
            if(arch=='USAD'):
                values=values[300:350]
            fname = file.split(".npy")[0]
            
            val_scores[arch][fname] = values

        if "csv" in file:
            fpath = os.path.join(subdir, file)
           
            df = pd.read_csv(fpath, header = None)
            values = [x[0] for x in df.values]

            fname = file.split(".csv")[0]
            val_scores[arch][fname] = values





In [15]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve

def plot_roc_curve(true_y, y_prob):
    """
    plots the roc curve based of the probabilities
    """

    fpr, tpr, thresholds = roc_curve(true_y, y_prob)
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')

In [16]:
for subdir, dirs, files in os.walk("SCORE_SKAB"):
    for file in files:
        arch = subdir.split("\\")[1]
        if arch not in scores_map:
            scores_map[arch] = dict()
            
        if arch not in rocauc_map:
            rocauc_map[arch] = dict()
            
        if "npy" not in file:
            fpath = os.path.join(subdir, file)
            df = pd.read_csv(fpath, header = None)
            values = [x[0] for x in df.values]
            
            fname = file.split(".csv")[0]
            
            scores_map[arch][fname] = values
            gt = gt_map[fname]
            
            rocauc_map[arch][fname] = compute_rocauc(values, gt)

        if "npy" in file:
            fpath = os.path.join(subdir, file)
            
            values = [x for x in np.load(fpath)]
            if(arch=='USAD'):
                values=values[300:350]
            fname = file.split(".npy")[0]
            
            scores_map[arch][fname] = values
            gt = gt_map[fname]
            
            rocauc_map[arch][fname] = compute_rocauc(values, gt)



In [17]:
print(rocauc_map['USAD'])

{'1': 0.9999900203584686, '10': 0.9981921184645428, '11': 0.9953666034718409, '12': 0.7648745519713261, '13': 0.9915024677527322, '14': 0.8416804249763402, '15': 0.8106165971256375, '16': 0.9827131201123406, '17': 0.8589091112181568, '18': 0.8932897096297228, '19': 0.9719347182461034, '2': 0.9575259066740923, '20': 0.9283474048249114, '21': 0.9903287073859741, '22': 0.7443790572608866, '23': 0.9388751375137514, '24': 0.7521807459381376, '25': 0.9543383359330468, '26': 0.7808497232865048, '27': 0.9586187691450849, '28': 0.9402847222222223, '29': 0.9566847204421107, '3': 0.8205479664459708, '30': 0.8289225224190295, '31': 0.8917217500786907, '32': 0.7967908718131573, '33': 0.9623593530239098, '4': 0.9396971960078757, '5': 0.8244330694601512, '6': 0.9715826556405769, '7': 0.826190392522051, '8': 0.9959729782908113, '9': 0.9961537160447888}


In [18]:
rocauc_map_mean = dict()

In [20]:
for rme in rocauc_map:
    rocauc_map_mean[rme] = np.mean(np.array(list(rocauc_map[rme].values())))

In [21]:
rocauc_map_mean

{'CONV-AE': 0.9422094169622846,
 'DENSE-AE': 0.856511575994107,
 'ELM': 0.8959215619853937,
 'GT': 1.0,
 'LSTM': 0.8769714848425851,
 'LSTM-AE': 0.8704516110898716,
 'REENC': 0.9161168216356255,
 'USAD': 0.9050259135060893,
 'VAE': 0.9228964874569958}

In [22]:
array=['CONV-AE','DENSE-AE','LSTM','VAE','REENC','LSTM-AE','ELM','USAD']



In [18]:
thresholds_dict = {
    'MV': None,
    'MAD': {0.5: None, 1.0: None, 1.5: None, 2.0: None},
    'IQR': {0.5: None, 1.0: None, 1.5: None, 2.0: None},
    'SD': {0.5: None, 1.0: None, 1.5: None, 2.0: None}
}

In [19]:
meth_thres_dict = {}

In [20]:
for meth in val_scores:
    all_files_scores = sum(val_scores[meth].values(), [])
    
    iqr_scores = [compute_threshold(all_files_scores, 'IQR', th_factor) for th_factor in [0.5, 1.0, 1.5, 2.0]]
    thresholds_dict['IQR'][0.5] = iqr_scores[0]
    thresholds_dict['IQR'][1.0] = iqr_scores[1]
    thresholds_dict['IQR'][1.5] = iqr_scores[2]
    thresholds_dict['IQR'][2.0] = iqr_scores[3]
    
    mad_scores = [compute_threshold(all_files_scores, 'MAD', th_factor) for th_factor in [0.5, 1.0, 1.5, 2.0]]
    thresholds_dict['MAD'][0.5] = mad_scores[0]
    thresholds_dict['MAD'][1.0] = mad_scores[1]
    thresholds_dict['MAD'][1.5] = mad_scores[2]
    thresholds_dict['MAD'][2.0] = mad_scores[3]
    
    
    sd_scores = [compute_threshold(all_files_scores, 'SD', th_factor) for th_factor in [0.5, 1.0, 1.5, 2.0]]
    thresholds_dict['SD'][0.5] = sd_scores[0]
    thresholds_dict['SD'][1.0] = sd_scores[1]
    thresholds_dict['SD'][1.5] = sd_scores[2]
    thresholds_dict['SD'][2.0] = sd_scores[3]
    
    mv_scores = compute_threshold(all_files_scores, 'MV', None)
    thresholds_dict['MV'] = mv_scores
    
    meth_thres_dict[meth] = thresholds_dict.copy()

In [21]:
meth_thres_dict

{'LSTM': {'MV': 0.2404910352995658,
  'MAD': {0.5: 0.00038921747887216043,
   1.0: 0.0004812285037653055,
   1.5: 0.0005732395286584506,
   2.0: 0.0006652505535515957},
  'IQR': {0.5: 0.0006601860659429803,
   1.0: 0.0008134559320751578,
   1.5: 0.0009667257982073352,
   2.0: 0.0011199956643395126},
  'SD': {0.5: 0.0008419341174885631,
   1.0: 0.001182127627544105,
   1.5: 0.001522321137599647,
   2.0: 0.001862514647655189}},
 'USAD': {'MV': 0.12259942,
  'MAD': {0.5: 0.00038921747887216043,
   1.0: 0.0004812285037653055,
   1.5: 0.0005732395286584506,
   2.0: 0.0006652505535515957},
  'IQR': {0.5: 0.0006601860659429803,
   1.0: 0.0008134559320751578,
   1.5: 0.0009667257982073352,
   2.0: 0.0011199956643395126},
  'SD': {0.5: 0.0008419341174885631,
   1.0: 0.001182127627544105,
   1.5: 0.001522321137599647,
   2.0: 0.001862514647655189}},
 'VAE': {'MV': 0.1698018496443232,
  'MAD': {0.5: 0.00038921747887216043,
   1.0: 0.0004812285037653055,
   1.5: 0.0005732395286584506,
   2.0: 0.00

In [22]:
f1_map = dict()

In [23]:
def compute_tpfpfn(pred_scores, gt, thresh, verbose= False):
    if verbose:
        print(thresh)
        
    preds = (pred_scores>thresh)*1
    if len(preds) < len(gt):
        preds = [0]*(len(gt) - len(preds)) + list(preds)
             
    cm = confusion_matrix(preds, gt)
    
    TP = cm[0][0]
    FP = cm[0][1]
    FN = cm[1][0]
    TN = cm[1][1]
        
    return TP, FP, FN, TN

In [24]:
compute_tpfpfn(np.array([0.8, 0.7, 0.6, 0.5]), np.array([0, 1, 1, 1]), 0.9)

(1, 3, 0, 0)

In [25]:
for subdir, dirs, files in os.walk("SCORE_SKAB"):
    for file in files:
        arch = subdir.split("/")[1]
        
        if arch not in f1_map:
            f1_map[arch] = dict()
            
        if "npy" not in file:
            fpath = os.path.join(subdir, file)
            df = pd.read_csv(fpath, header = None)

            values = [x[0] for x in df.values]
                
            fname = file.split(".csv")[0]
            
            gt = gt_map[fname]
            
            
            for thr_method in meth_thres_dict[arch]:
                if thr_method not in f1_map[arch]:
                    f1_map[arch][thr_method] = dict()
                
                for th_factor in [0.5, 1.0, 1.5, 2.0]:
                    if th_factor not in f1_map[arch][thr_method]:                    
                        f1_map[arch][thr_method][th_factor] = dict()
                    
                    if thr_method == 'MV':
                        thresh = meth_thres_dict[arch][thr_method]
                    else:
                        thresh = meth_thres_dict[arch][thr_method][th_factor]
                
                    if fname not in f1_map[arch][thr_method][th_factor]:
                        f1_map[arch][thr_method][th_factor][fname] = dict()
                       
                    verbose = False
                    if arch == "CONV-AE" and thr_method == "MV" and fname == "1":
                        verbose = True
                        
                    f1_map[arch][thr_method][th_factor][fname]['TP'], f1_map[arch][thr_method][th_factor][fname]['FP'], f1_map[arch][thr_method][th_factor][fname]['FN'], f1_map[arch][thr_method][th_factor][fname]['TN'] = compute_tpfpfn(values, gt, thresh, verbose)
                    
                    if arch == "CONV-AE" and thr_method == "MV" and fname == "1":
                        print(f1_map[arch][thr_method][th_factor][fname]['TP'], f1_map[arch][thr_method][th_factor][fname]['FP'], f1_map[arch][thr_method][th_factor][fname]['FN'])
                    

0.014434438282246857
557 45 0
0.014434438282246857
557 45 0
0.014434438282246857
557 45 0
0.014434438282246857
557 45 0


In [26]:
f1_map

{'CONV-AE': {'MV': {0.5: {'1': {'TP': 557, 'FP': 45, 'FN': 0, 'TN': 143},
    '10': {'TP': 729, 'FP': 1, 'FN': 16, 'TN': 401},
    '11': {'TP': 730, 'FP': 2, 'FN': 13, 'TN': 345},
    '12': {'TP': 508, 'FP': 236, 'FN': 236, 'TN': 167},
    '13': {'TP': 732, 'FP': 10, 'FN': 11, 'TN': 391},
    '14': {'TP': 575, 'FP': 132, 'FN': 171, 'TN': 269},
    '15': {'TP': 738, 'FP': 402, 'FN': 5, 'TN': 0},
    '16': {'TP': 745, 'FP': 401, 'FN': 0, 'TN': 0},
    '17': {'TP': 742, 'FP': 104, 'FN': 0, 'TN': 295},
    '18': {'TP': 630, 'FP': 0, 'FN': 111, 'TN': 399},
    '19': {'TP': 741, 'FP': 81, 'FN': 0, 'TN': 318},
    '2': {'TP': 570, 'FP': 53, 'FN': 171, 'TN': 533},
    '20': {'TP': 740, 'FP': 111, 'FN': 0, 'TN': 288},
    '21': {'TP': 746, 'FP': 58, 'FN': 0, 'TN': 346},
    '22': {'TP': 738, 'FP': 337, 'FN': 0, 'TN': 0},
    '23': {'TP': 744, 'FP': 316, 'FN': 0, 'TN': 88},
    '24': {'TP': 732, 'FP': 348, 'FN': 14, 'TN': 1},
    '25': {'TP': 751, 'FP': 401, 'FN': 0, 'TN': 2},
    '26': {'TP': 7

In [27]:
import pandas as pd

def flatten_dict(nested_dict):
    res = {}
    if isinstance(nested_dict, dict):
        for k in nested_dict:
            flattened_dict = flatten_dict(nested_dict[k])
            for key, val in flattened_dict.items():
                key = list(key)
                key.insert(0, k)
                res[tuple(key)] = val
    else:
        res[()] = nested_dict
    return res


def nested_dict_to_df(values_dict):
    flat_dict = flatten_dict(values_dict)
    df = pd.DataFrame.from_dict(flat_dict, orient="index")
    df.index = pd.MultiIndex.from_tuples(df.index)
    df = df.unstack(level=-1)
    df.columns = df.columns.map("{0[1]}".format)
    return df

In [28]:
results_df = nested_dict_to_df(f1_map)
results_df[250:300]

FN   FP   TN   TP
CONV-AE MAD 2.0 27  677    0  405   12
                28  732    0  400   12
                29  734    0  402   12
                3   727    0  451   12
                30  719    0  394   12
                31  718    0  333   12
                32  722    0  395   12
                33  588    0  395   12
                4   727    0  309   12
                5   646    0  265   12
                6   591    0  302   12
                7   727    0  398   12
                8   784    0  395   12
                9   733    0  410   12
        MV  0.5 1     0   45  143  557
                10   16    1  401  729
                11   13    2  345  730
                12  236  236  167  508
                13   11   10  391  732
                14  171  132  269  575
                15    5  402    0  738
                16    0  401    0  745
                17    0  104  295  742
                18  111    0  399  630
                19    0   81  318  741
                2   171   53  533  570
                20    0  111  288  740
                21    0   58  346  746
                22    0  337    0  738
                23    0  316   88  744
                24   14  348    1  732
                25    0  401    2  751
                26    0  403    2  749
                27   58   83  322  631
                28    0  271  129  744
                29    0   90  312  746
                3   125  280  171  614
                30    5  380   14  726
                31    0  325    8  730
                32    0  389    6  734
                33    0  100  295  600
                4    51   72  237  688
                5     0  265    0  658
                6    35   16  286  568
                7     0  398    0  739
                8     0  391    4  796
                9   180    1  409  565
            1.0 1     0   45  143  557
                10   16    1  401  729
                11   13    2  345  730

In [29]:
results_df = results_df.reset_index().groupby(["level_0", "level_1", "level_2"]).sum()

In [30]:
results_df["F1"] = results_df["TP"]/(results_df["TP"] + 0.5*(results_df["FP"] + results_df["FN"]))

In [36]:
results_df[:50]

FN    FP     TN     TP        F1
level_0  level_1 level_2                                     
CONV-AE  IQR     0.5      23542     0  12683    396  0.032547
                 1.0      23539     0  12683    399  0.032790
                 1.5      23528     0  12683    410  0.033678
                 2.0      23482     0  12683    456  0.037386
         MAD     0.5      23542     0  12683    396  0.032547
                 1.0      23542     0  12683    396  0.032547
                 1.5      23542     0  12683    396  0.032547
                 2.0      23542     0  12683    396  0.032547
         MV      0.5       1202  6502   6181  22736  0.855123
                 1.0       1202  6502   6181  22736  0.855123
                 1.5       1202  6502   6181  22736  0.855123
                 2.0       1202  6502   6181  22736  0.855123
         SD      0.5      23538     0  12683    400  0.032870
                 1.0      23453     0  12683    485  0.039717
                 1.5      22995     0  12683    943  0.075801
                 2.0      21943    12  12671   1995  0.153787
DENSE-AE IQR     0.5      23542     0  12683    396  0.032547
                 1.0      23542     0  12683    396  0.032547
                 1.5      23542     0  12683    396  0.032547
                 2.0      23542     0  12683    396  0.032547
         MAD     0.5      23542     0  12683    396  0.032547
                 1.0      23542     0  12683    396  0.032547
                 1.5      23542     0  12683    396  0.032547
                 2.0      23542     0  12683    396  0.032547
         MV      0.5       1149  7915   4768  22789  0.834120
                 1.0       1149  7915   4768  22789  0.834120
                 1.5       1149  7915   4768  22789  0.834120
                 2.0       1149  7915   4768  22789  0.834120
         SD      0.5      23542     0  12683    396  0.032547
                 1.0      23542     0  12683    396  0.032547
                 1.5      23542     0  12683    396  0.032547
                 2.0      23542     0  12683    396  0.032547
ELM      IQR     0.5       4416  1719  10964  19522  0.864207
                 1.0       4324  1808  10875  19614  0.864815
                 1.5       4298  1893  10790  19640  0.863847
                 2.0       4260  1956  10727  19678  0.863600
         MAD     0.5       4688  1596  11087  19250  0.859682
                 1.0       4470  1630  11053  19468  0.864553
                 1.5       4446  1677  11006  19492  0.864256
                 2.0       4412  1722  10961  19526  0.864250
         MV      0.5       3139  2720   9963  20799  0.876541
                 1.0       3139  2720   9963  20799  0.876541
                 1.5       3139  2720   9963  20799  0.876541
                 2.0       3139  2720   9963  20799  0.876541
         SD      0.5       4321  1826  10857  19617  0.864547
                 1.0       4197  1975  10708  19741  0.864809
                 1.5       4098  2035  10648  19840  0.866130
                 2.0       3936  2097  10586  20002  0.868953
LSTM     IQR     0.5      23738     0  12683    200  0.016571
                 1.0      23737     0  12683    201  0.016654

In [32]:
sorted_scores = []
sorted_gt = []

In [33]:
tp, fp, fn, tn = 0, 0, 0, 0

for i in range(33):
    si = str(i+1)
    sorted_scores.append(scores_map['CONV-AE'][si])
    sorted_gt.append(gt_map[si])
    
    tpp, fpp, fnp, tnp = compute_tpfpfn(np.array(scores_map['CONV-AE'][si]), gt_map[si], 0.014434438282246857)
    tp += tpp
    fp += fpp
    fn += fnp
    tn += tnp

In [34]:
fn, fp, tp, tn

(1202, 6502, 22736, 6181)

In [35]:
tp/(tp+0.5*(fn+fp))

0.8551226117045284